In [1]:
import torch
from torch.utils import data
import sys
sys.path.append("..")

import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

## Dummy Data Generation

### Parameters
Lets generate some random data to demonstrate our setup

In [2]:
N_SAMPLES = 99 # Number of training samples to generate
N_ATOMS = 5 # E.g. number of agents in a simulation
N_STEPS = 10 # Length of time series (e.g. trajectory length)
N_FEATURES = 3 # Number of features per atom and timestep

### Generate Random Data

In [3]:
data_loaders = dict(
    train_loader= data.DataLoader(data.TensorDataset(torch.rand(N_SAMPLES, N_ATOMS, N_STEPS, N_FEATURES))),
    valid_loader = data.DataLoader(data.TensorDataset(torch.rand(N_SAMPLES, N_ATOMS, N_STEPS, N_FEATURES))),
    test_loader = data.DataLoader(data.TensorDataset(torch.rand(N_SAMPLES, N_ATOMS, N_STEPS, N_FEATURES)))
)

## Model Definition
### Model Hyperparameters

In [4]:
N_EDGE_TYPES = 3
DIM_ENCODER_HIDDEN = 20
DIM_DECODER_HIDDEN = 20

In [5]:
from src.model.modules import MLPEncoder, RNNDecoder
from src.trainer import generate_config
from src.trainer import Trainer


def objective_function(device_id, n_edge_types, dim_encoder_hidden):
    encoder = MLPEncoder(N_STEPS * N_FEATURES, DIM_ENCODER_HIDDEN, N_EDGE_TYPES)
    decoder = RNNDecoder(n_in_node=N_FEATURES, edge_types=N_EDGE_TYPES, n_hid=DIM_DECODER_HIDDEN)
    
    
    config = generate_config(
        n_edges=n_edge_types,
        n_atoms=N_ATOMS,
        epochs=1,
        early_stopping_patience=2,
        temp=0.5,
        gpu_id=device_id  
    )
    
    trainer = Trainer(encoder=encoder,
                decoder=decoder,
                data_loaders=data_loaders,
                config=config)
    trainer.train()
    return trainer._val_loss(0)['val_loss']

In [6]:
import skopt
import torch
from skopt.space import Integer
from src.hpo import HPO

space = [Integer(1, 5, name='n_edge_types'),
        Integer(20, 300, name='dim_encoder_hidden')]

hpo = HPO(objective_function=objective_function,
          search_space=space,
          n_iters=5,
          gpu_ids=[],
          n_initial_points=2
         )

res = hpo.tune(return_best_n=2)


print("---------------------")
for n, (params, result) in enumerate(res):
    conf_str = str([(space[i].name, val) for (i, val) in enumerate(params)])
    s = f"Score of {result} achieved by configuration: {conf_str}"
    print(s)

INFO:trainer:    epoch          : 0
INFO:trainer:    loss           : 712.834127108256
INFO:trainer:    val_loss       : 18.59517618622443


INFO:trainer:    epoch          : 0
INFO:trainer:    loss           : 671.448364575704
INFO:trainer:    val_loss       : 15.828539983190671


INFO:trainer:    epoch          : 0
INFO:trainer:    loss           : 674.3302281697592
INFO:trainer:    val_loss       : 15.390117076912311


INFO:trainer:    epoch          : 0
INFO:trainer:    loss           : 708.494994799296
INFO:trainer:    val_loss       : 18.188052553119082


INFO:trainer:    epoch          : 0
INFO:trainer:    loss           : 691.9259936014811
INFO:trainer:    val_loss       : 16.722219785054524


---------------------
Score of 15.382021923257847 achieved by configuration: [('n_edge_types', 4), ('dim_encoder_hidden', 282)]
Score of 15.83726188389942 achieved by configuration: [('n_edge_types', 4), ('dim_encoder_hidden', 267)]
